In [ ]:
from datetime import datetime, timedelta

In [ ]:
today = datetime.now()
date_60_days_ago = today - timedelta(days=60)
print(f"当前日期: {today.strftime('%Y-%m-%d')}")
print(f"60天前的日期: {date_60_days_ago.strftime('%Y-%m-%d')}")

In [ ]:
def calculate_price_60_days_ago(current_price, change_percentage):
    # 计算60天前的收盘价
    price_60_days_ago = current_price / (1 + change_percentage / 100)
    return price_60_days_ago

In [ ]:
current_price = 1531.62  # 今日收盘价
change_percentage = 7.69  # 60日涨跌幅

In [ ]:
price_60_days_ago = calculate_price_60_days_ago(current_price, change_percentage)
print(f"60天前的收盘价为: {price_60_days_ago:.2f}元")

In [1]:
import os
import django

# 设置Django的环境变量
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "x_data.settings")
django.setup()

In [2]:
from django.core.cache import cache

In [3]:
import pickle

In [4]:
dataset = pickle.loads(cache.get("2023_stock_history"))

In [5]:
print(type(dataset))

<class 'pandas.core.frame.DataFrame'>


In [7]:
import pandas as pd

In [33]:
# 给定日期
given_date = '2023-07-03'

# 将trading_date列转换为datetime类型
dataset['trading_date'] = pd.to_datetime(dataset['trading_date'])

# 筛选出在给定日期之前的记录
# df_filtered = dataset[dataset['trading_date'] <= given_date]
df_filtered = dataset[(dataset['trading_date'] < given_date) | (dataset['trading_date'] == given_date)]

# 确保每只股票在给定日期之前有记录
valid_stocks = df_filtered.groupby('stock_code').filter(lambda x: len(x) > 0)['stock_code'].unique()

In [35]:
def calculate_return(stock_data):
    if len(stock_data) < 10:
        return None
    # 获取10日之前的收盘价
    closing_price_10_days_ago = stock_data['closing_price'].iloc[-10]
    # 当前收盘价
    current_closing_price = stock_data['closing_price'].iloc[-1]
    # 当前日期
    current_date = stock_data['trading_date'].iloc[-1]
    # 10天前的日期
    date_10_days_ago = stock_data['trading_date'].iloc[-10]
    # 计算涨跌幅
    return round((current_closing_price - closing_price_10_days_ago) / closing_price_10_days_ago * 100, 2), current_date, current_closing_price, date_10_days_ago, closing_price_10_days_ago

# 对每个有效的股票代码进行计算
result = df_filtered[df_filtered['stock_code'].isin(valid_stocks)].groupby('stock_code').apply(calculate_return).reset_index()

# 检查返回的列数并展开元组
if result.shape[1] == 2:  # 确保有2列（股票代码和返回值）
    result[['10_day_return', 'current_date', 'current_closing_price', 'date_10_days_ago', 'closing_price_10_days_ago']] = pd.DataFrame(result[0].tolist(), index=result.index)
    result = result.drop(columns=0)  # 删除原来的返回值列

# 删除缺失值
result = result.dropna()

# 按照涨跌幅从高到低排序，涨跌幅相同情况下按照股票代码正序排序
result = result.sort_values(by=['10_day_return', 'stock_code'], ascending=[False, True])

# 取前100条记录
result_top_100 = result.head(100)

# 输出结果
print(result_top_100)

     stock_code  10_day_return current_date  current_closing_price  \
5235     872931         124.37   2023-06-08                   8.01   
4267     603767          80.95   2023-07-03                  14.82   
2004     300503          67.34   2023-07-03                  19.93   
4968     688599          66.43   2023-01-11                  68.90   
3872     603042          65.86   2023-07-03                  22.64   
...         ...            ...          ...                    ...   
874      002356          18.52   2023-07-03                   4.80   
2510     301031          18.52   2023-07-03                 140.80   
2177     300680          18.48   2023-07-03                  22.50   
2700     301263          18.46   2023-07-03                  22.59   
602      002079          18.40   2023-07-03                  14.35   

     date_10_days_ago  closing_price_10_days_ago  
5235       2023-02-24                       3.57  
4267       2023-06-16                       8.19  
2004  

/var/folders/1c/t08y_l4d6j7g6lrplpp5k55m0000gn/T/ipykernel_53006/1648219285.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df_filtered[df_filtered['stock_code'].isin(valid_stocks)].groupby('stock_code').apply(calculate_return).reset_index()


In [21]:
# 计算60日涨跌幅
def calculate_return(stock_data):
    if len(stock_data) < 10:
        return None
    # 获取x日之前的收盘价
    closing_price_x_days_ago = stock_data['closing_price'].iloc[-10]
    # 当前收盘价
    current_closing_price = stock_data['closing_price'].iloc[-1]
    # 当前日期
    current_date = stock_data['trading_date'].iloc[-1]
    # x天前的日期
    date_x_days_ago = stock_data['trading_date'].iloc[-10]
    # 计算涨跌幅
    return round((current_closing_price - closing_price_x_days_ago) / closing_price_x_days_ago * 100,
                 2), current_date, current_closing_price, date_x_days_ago, closing_price_x_days_ago


# 对每个股票代码进行计算
result = df_filtered.groupby('stock_code').apply(calculate_return, include_groups=False).reset_index()
result.columns = ['stock_code', 'x_day_return', 'current_date', 'current_closing_price', 'date_x_days_ago',
                  'closing_price_x_days_ago']

# 删除缺失值
result = result.dropna()

# 按照涨跌幅从高到低排序，涨跌幅相同情况下按照股票代码正序排序
result = result.sort_values(by=['x_day_return', 'stock_code'], ascending=[False, True])

# 输出结果
result_top_100 = result.head(100)
print(result_top_100)
print("=" * 10)
result_last_100 = result.tail(100)
print(result_last_100)

ValueError: Length mismatch: Expected axis has 2 elements, new values have 6 elements